### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import sys
!{sys.executable} -m pip install pytz

### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata'
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

### Crea primera tabla historica preliminar a nivel de usuarios

In [4]:
# Tabla historica preliminar para capturar latitud y longitud frecuente
fecha_inicio = '02-01-2020'
fecha_final = '03-31-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)
fechas

DatetimeIndex(['2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04',
               '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08',
               '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12',
               '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16',
               '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20',
               '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24',
               '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
               '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03',
               '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07',
               '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11',
               '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15',
               '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-19',
               '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-23',
               '2020-03-24', '2020-03-25', '2020-03-26', '2020-03-27',
      

In [5]:
historico_usuarios_deltas = f'historico_usuarios_preliminar'
start_time = time.time()
for i,fecha in enumerate(fechas):
    time_fecha = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    tabla_usuarios_fecha = f'todos_usuarios_delta_{codigo_fecha}'
    if i == 0 and (day == '01' and fecha.month == 2):
        query = f'''
                    CREATE TABLE {historico_usuarios_deltas}
                    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/{historico_usuarios_deltas}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY',
                        partitioned_by = ARRAY['iso_country_code']) AS
                    SELECT *
                    FROM {tabla_usuarios_fecha}
                    WHERE lat_hogar IS NOT NULL
                '''
    else:
        query = f'''
                    INSERT INTO {historico_usuarios_deltas} SELECT * FROM {tabla_usuarios_fecha} WHERE lat_hogar IS NOT NULL 
                '''
    cursor.execute(query)
    tiempo = (time.time()-time_fecha)/60
    print(f"{codigo_fecha} insertada, {int(tiempo)} minutos")
print((time.time()-start_time)/60)

201
201 insertada, 0 minutos
202
202 insertada, 0 minutos
203
203 insertada, 0 minutos
204
204 insertada, 0 minutos
205
205 insertada, 0 minutos
206
206 insertada, 0 minutos
207
207 insertada, 0 minutos
208
208 insertada, 0 minutos
209
209 insertada, 0 minutos
210
210 insertada, 0 minutos
211
211 insertada, 0 minutos
212
212 insertada, 0 minutos
213
213 insertada, 0 minutos
214
214 insertada, 0 minutos
215
215 insertada, 0 minutos
216
216 insertada, 0 minutos
217
217 insertada, 0 minutos
218
218 insertada, 0 minutos
219
219 insertada, 0 minutos
220
220 insertada, 0 minutos
221
221 insertada, 0 minutos
222
222 insertada, 0 minutos
223
223 insertada, 0 minutos
224
224 insertada, 0 minutos
225
225 insertada, 0 minutos
226
226 insertada, 0 minutos
227
227 insertada, 0 minutos
228
228 insertada, 0 minutos
229
229 insertada, 0 minutos
301
301 insertada, 0 minutos
302
302 insertada, 0 minutos
303
303 insertada, 0 minutos
304
304 insertada, 0 minutos
305
305 insertada, 0 minutos
306
306 insert

In [6]:
query = f'''MSCK REPAIR TABLE {historico_usuarios_deltas}'''
cursor.execute(query)